# Preparación de datos de test

In [22]:
import pickle as pckl

In [23]:
input_path = "/home/ubuntu/tfm/TrainYourOwnYOLO/Data/Source_Images/Training_Images/vott-csv-export-new-parsed/data_train.txt"
output_path = "/home/ubuntu/tfm/TrainYourOwnYOLO/Data/Source_Images/Training_Images/vott-csv-export-new-parsed/data_test.pckl"
mapper = {0:'Panel', 1:'Dedo'}

In [24]:
rows = []
with open(input_path) as fd:
    for item in fd:
        filename_and_boxes = item.rstrip('\n').split(' ')
        filename = filename_and_boxes[0]
        boxes = filename_and_boxes[1:]
        d = {'filename': filename, 'object':[]}
        for box in boxes:
            box = box.split(',')
            d['object'].append({'xmin':int(box[0]), 'ymin':int(box[1]), 'xmax': int(box[2]), 'ymax': int(box[3]), 'name': mapper[int(box[4])]})
        rows.append(d)

In [25]:
pckl.dump(rows, open(output_path, 'wb'))

In [26]:
rows[0]

{'filename': '/home/ubuntu/tfm/TrainYourOwnYOLO/Data/Source_Images/Training_Images/vott-csv-export-new-parsed/aug_VID_20201230_201640.mp4#t=1.666667.jpg',
 'object': [{'xmin': 0,
   'ymin': 523,
   'xmax': 595,
   'ymax': 710,
   'name': 'Panel'},
  {'xmin': 703, 'ymin': 580, 'xmax': 1080, 'ymax': 727, 'name': 'Panel'}]}

# Dependencias

In [54]:
import argparse
import json
import pickle as pckl
import numpy as np
import os
import cv2
import pandas as pd
from PIL import Image
from scipy.special import expit
from yolo3.yolo import YOLO
from tqdm import tqdm

/home/ubuntu/miniconda3/envs/tfm/lib/python3.7/site-packages/tensorflow/python/data/ops/iterator_ops.py:546: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  class IteratorBase(collections.Iterator, trackable.Trackable,
/home/ubuntu/miniconda3/envs/tfm/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py:106: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  class DatasetV2(collections.Iterable, tracking_base.Trackable,


## Cargar modelo

In [6]:
def load_model(model_path, classes_path, anchors_path):

    yolo = YOLO(
        **{
            "model_path": model_path,
            "anchors_path": anchors_path,
            "classes_path": classes_path,
            "score": 0.5,
            "gpu_num": 1,
            "model_image_size": (416, 416),
        }
    )

    return yolo

## Bounding boxes

In [7]:
class BoundBox:
    def __init__(self, xmin, ymin, xmax, ymax, c = None, classes = None):
        self.xmin = xmin
        self.ymin = ymin
        self.xmax = xmax
        self.ymax = ymax
        
        self.c       = c
        self.classes = classes

        self.label = -1
        self.score = -1

    def get_label(self):
        if self.label == -1:
            self.label = np.argmax(self.classes)
        
        return self.label
    
    def get_score(self):
        if self.score == -1:
            self.score = self.classes[self.get_label()]
            
        return self.score 

def _interval_overlap(interval_a, interval_b):
    x1, x2 = interval_a
    x3, x4 = interval_b

    if x3 < x1:
        if x4 < x1:
            return 0
        else:
            return min(x2,x4) - x1
    else:
        if x2 < x3:
             return 0
        else:
            return min(x2,x4) - x3  
        
def bbox_iou(box1, box2):
    intersect_w = _interval_overlap([box1.xmin, box1.xmax], [box2.xmin, box2.xmax])
    intersect_h = _interval_overlap([box1.ymin, box1.ymax], [box2.ymin, box2.ymax])  
    
    intersect = intersect_w * intersect_h

    w1, h1 = box1.xmax-box1.xmin, box1.ymax-box1.ymin
    w2, h2 = box2.xmax-box2.xmin, box2.ymax-box2.ymin
    
    union = w1*h1 + w2*h2 - intersect
    
    return float(intersect) / union

## Generador de lotes

In [8]:
class BatchGenerator():
    def __init__(self, instances, anchors, labels, batch_size=1, shuffle=True):
        self.instances          = instances
        self.batch_size         = batch_size
        self.labels             = labels
        self.anchors            = [BoundBox(0, 0, anchors[2*i], anchors[2*i+1]) for i in range(len(anchors)//2)]

        if shuffle:
            np.random.shuffle(self.instances)      
            
    def num_classes(self):
        return len(self.labels)

    def size(self):
        return len(self.instances)    

    def get_anchors(self):
        anchors = []

        for anchor in self.anchors:
            anchors += [anchor.xmax, anchor.ymax]

        return anchors

    def load_annotation(self, i):
        annots = []

        for obj in self.instances[i]['object']:
            annot = [obj['xmin'], obj['ymin'], obj['xmax'], obj['ymax'], self.labels.index(obj['name'])]
            annots += [annot]

        if len(annots) == 0: annots = [[]]

        return np.array(annots)

    def load_image(self, i):
        return cv2.imread(self.instances[i]['filename'])    

## Detection

In [9]:
def do_nms(boxes, nms_thresh):
    if len(boxes) > 0:
        nb_class = len(boxes[0].classes)
    else:
        return
        
    for c in range(nb_class):
        sorted_indices = np.argsort([-box.classes[c] for box in boxes])

        for i in range(len(sorted_indices)):
            index_i = sorted_indices[i]

            if boxes[index_i].classes[c] == 0: continue

            for j in range(i+1, len(sorted_indices)):
                index_j = sorted_indices[j]

                if bbox_iou(boxes[index_i], boxes[index_j]) >= nms_thresh:
                    boxes[index_j].classes[c] = 0
                    
def get_yolo_boxes(model, images, net_h, net_w, nms_thresh):
    batch_output, data = model.detect_image(Image.fromarray(images[0].astype('uint8')))
    boxes = []
    
    for bo in batch_output:
        b = [0]*2
        b[bo[4]] = bo[5]
        box = bo[:4] + [bo[5]] + [b]
        boxes.append(BoundBox(box[0], box[1], box[2], box[3], box[4], box[5]))

    # image_h, image_w, _ = images[0].shape
    # correct_yolo_boxes(boxes, image_h, image_w, net_h, net_w)

    do_nms(boxes, nms_thresh)
    return [boxes]


In [10]:
def detection(model, generator, nms_thresh=0.5, net_h=416, net_w=416):  
    # gather all detections and annotations
    all_detections     = [[None for i in range(generator.num_classes())] for j in range(generator.size())]
    all_annotations    = [[None for i in range(generator.num_classes())] for j in range(generator.size())]

    for i in range(generator.size()):
        raw_image = [generator.load_image(i)]

        # make the boxes and the labels
        pred_boxes = get_yolo_boxes(model, raw_image, net_h, net_w, nms_thresh)[0]

        score = np.array([box.get_score() for box in pred_boxes])
        pred_labels = np.array([box.label for box in pred_boxes])        
        
        if len(pred_boxes) > 0:
            pred_boxes = np.array([[box.xmin, box.ymin, box.xmax, box.ymax, box.get_score()] for box in pred_boxes]) 
        else:
            pred_boxes = np.array([[]])  
        
        # sort the boxes and the labels according to scores
        score_sort = np.argsort(-score)
        pred_labels = pred_labels[score_sort]
        pred_boxes  = pred_boxes[score_sort]
        
        # copy detections to all_detections
        for label in range(generator.num_classes()):
            all_detections[i][label] = pred_boxes[pred_labels == label, :]

        annotations = generator.load_annotation(i)
        
        # copy detections to all_annotations
        for label in range(generator.num_classes()):
            all_annotations[i][label] = annotations[annotations[:, 4] == label, :4].copy()
            
    return all_detections, all_annotations

## Evaluation

In [11]:
def compute_overlap(a, b):
    """
    Code originally from https://github.com/rbgirshick/py-faster-rcnn.
    Parameters
    ----------
    a: (N, 4) ndarray of float
    b: (K, 4) ndarray of float
    Returns
    -------
    overlaps: (N, K) ndarray of overlap between boxes and query_boxes
    """
    area = (b[:, 2] - b[:, 0]) * (b[:, 3] - b[:, 1])

    iw = np.minimum(np.expand_dims(a[:, 2], axis=1), b[:, 2]) - np.maximum(np.expand_dims(a[:, 0], 1), b[:, 0])
    ih = np.minimum(np.expand_dims(a[:, 3], axis=1), b[:, 3]) - np.maximum(np.expand_dims(a[:, 1], 1), b[:, 1])

    iw = np.maximum(iw, 0)
    ih = np.maximum(ih, 0)

    ua = np.expand_dims((a[:, 2] - a[:, 0]) * (a[:, 3] - a[:, 1]), axis=1) + area - iw * ih

    ua = np.maximum(ua, np.finfo(float).eps)

    intersection = iw * ih

    return intersection / ua  

def compute_ap(recall, precision):
    """ Compute the average precision, given the recall and precision curves.
    Code originally from https://github.com/rbgirshick/py-faster-rcnn.

    # Arguments
        recall:    The recall curve (list).
        precision: The precision curve (list).
    # Returns
        The average precision as computed in py-faster-rcnn.
    """
    # correct AP calculation
    # first append sentinel values at the end
    mrec = np.concatenate(([0.], recall, [1.]))
    mpre = np.concatenate(([0.], precision, [0.]))

    # compute the precision envelope
    for i in range(mpre.size - 1, 0, -1):
        mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

    # to calculate area under PR curve, look for points
    # where X axis (recall) changes value
    i = np.where(mrec[1:] != mrec[:-1])[0]

    # and sum (\Delta recall) * prec
    ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap  

In [45]:
def evaluation(all_detections, all_annotations, generator, iou_threshold=0.5):
    average_precisions = []
    
    for label in range(generator.num_classes()):
        false_positives = np.zeros((0,))
        true_positives  = np.zeros((0,))
        scores          = np.zeros((0,))
        num_annotations = 0.0

        for i in range(generator.size()):
            detections           = all_detections[i][label]
            annotations          = all_annotations[i][label]
            num_annotations     += annotations.shape[0]
            detected_annotations = []

            for d in detections:
                scores = np.append(scores, d[4])

                if annotations.shape[0] == 0: # Si no hay anotación de esa detección es un falso positivo
                    false_positives = np.append(false_positives, 1)
                    true_positives  = np.append(true_positives, 0)
                    continue

                overlaps            = compute_overlap(np.expand_dims(d, axis=0), annotations) # IOU, tiene el consideración todas las anotaciones
                assigned_annotation = np.argmax(overlaps, axis=1) # Se queda con la anotación que maximiza el IOU
                max_overlap         = overlaps[0, assigned_annotation] # Se queda con el valor del IOU se esta anotación

                if max_overlap >= iou_threshold and assigned_annotation not in detected_annotations: # Comprueba si esa anotación no ha sido ya asignada a una detección (además de comprobar que el IOU supera un cierto umbral). Las detecciones están ordenadas por score descendente por lo que se quedaría primero la que tiene mayor score (aunque luego pueda tener menor IoU).
                    false_positives = np.append(false_positives, 0)
                    true_positives  = np.append(true_positives, 1)
                    detected_annotations.append(assigned_annotation) # Guarda la anotación para que no pueda volver a ser usada
                else: # IOU por debajo del umbral o anotación ya utilizada
                    false_positives = np.append(false_positives, 1)
                    true_positives  = np.append(true_positives, 0) 

        # no annotations -> AP for this class is 0 (is this correct?)
        if num_annotations == 0:
            average_precisions[label] = 0
            continue

        # sort by score (Esto lo hace para ser consistente con los vectores de anotación y detección)
        indices         = np.argsort(-scores)
        false_positives = false_positives[indices]
        true_positives  = true_positives[indices]
        annotations_pending = num_annotations - np.sum(true_positives)

        # compute false positives and true positives (Esto es lo mismo que sumar unos y ceros de cada una de los vectores pero se hace así para computar el AP)
        false_positives = np.cumsum(false_positives)
        true_positives  = np.cumsum(true_positives)

        # compute recall and precision (Y el F1)
        recall    = true_positives / num_annotations # Es lo mismo que dividir entre TP + FN porque la suma de ambas tiene que ser el número de anotaciones (se detecten o no)
        precision = true_positives / np.maximum(true_positives + false_positives, np.finfo(np.float64).eps)
        f1 = 2 * (precision * recall) / (precision + recall)

        # compute average precision
        average_precision  = compute_ap(recall, precision)
        average_precisions.append({'label': generator.labels[label], 'AP': average_precision, 'recall': recall[-1] if len(recall) else -1, 'precision': precision[-1] if len(precision) else -1, 'support': num_annotations, 'TP':true_positives[-1] if len(true_positives) else -1, 'FP': false_positives[-1] if len(false_positives) else -1, 'FN': annotations_pending})

    return average_precisions

In [13]:
def evaluation_v2(all_detections, all_annotations, generator, confidence_threshold = 0.5, iou_threshold=0.5):
    average_precisions = []
    
    for label in range(generator.num_classes()):
        false_positives = np.zeros((0,))
        true_positives  = np.zeros((0,))
        false_negatives = np.zeros((0,))
        true_negatives  = np.zeros((0,))
        scores          = np.zeros((0,))
        num_annotations = 0.0
        annotations_pending = 0.0

        for i in range(generator.size()):
            detections           = all_detections[i][label]
            annotations          = all_annotations[i][label]
            num_annotations     += annotations.shape[0]
            detected_annotations = []

            for d in detections:
                scores = np.append(scores, d[4])

                if annotations.shape[0] == 0: # Si no hay anotación
                    # index = int(not bool(label))
                    # all_annotations[i][index].shape[0] == 0
                    if d[4] < confidence_threshold: # El score no supera un umbral, verdadero negativo
                        false_positives = np.append(false_positives, 0)
                        true_positives  = np.append(true_positives, 0)
                        false_negatives = np.append(false_negatives, 0)
                        true_negatives  = np.append(true_negatives, 1) 
                    else: # El score es alto, falso positivo
                        false_positives = np.append(false_positives, 1)
                        true_positives  = np.append(true_positives, 0)
                        false_negatives = np.append(false_negatives, 0)
                        true_negatives  = np.append(true_negatives, 0)
                    continue
                    
                overlaps            = compute_overlap(np.expand_dims(d, axis=0), annotations) # IOU, tiene el consideración todas las anotaciones
                assigned_annotation = np.argmax(overlaps, axis=1) # Se queda con la anotación que maximiza el IOU
                max_overlap         = overlaps[0, assigned_annotation] # Se queda con el valor del IOU se esta anotación
                
                if assigned_annotation in detected_annotations:
                    false_positives = np.append(false_positives, 1)
                    true_positives  = np.append(true_positives, 0)
                    false_negatives = np.append(false_negatives, 0)
                    true_negatives  = np.append(true_negatives, 0)
                    continue
                
                if max_overlap >= iou_threshold and d[4] >= confidence_threshold: # Comprueba que el IOU supera un cierto umbral de igual modo que la precisión en la clasificación debe superar otro
                    false_positives = np.append(false_positives, 0)
                    true_positives  = np.append(true_positives, 1)
                    false_negatives = np.append(false_negatives, 0)
                    true_negatives  = np.append(true_negatives, 0)
                    detected_annotations.append(assigned_annotation)
                else: # IOU por debajo del umbral o precisión en la clasificación por debajo de su umbral
                    false_positives = np.append(false_positives, 1)
                    true_positives  = np.append(true_positives, 0)
                    false_negatives = np.append(false_negatives, 0)
                    true_negatives  = np.append(true_negatives, 0)
                    
            annotations_pending = (annotations_pending + annotations.shape[0]) - len(detected_annotations) 
            
        # no annotations -> AP for this class is 0 (is this correct?)
        if num_annotations == 0:
            average_precisions[label] = 0
            continue

        # sort by score (Esto lo hace para ser consistente con los vectores de anotación y detección)
        indices         = np.argsort(-scores)
        false_positives = false_positives[indices]
        true_positives  = true_positives[indices]
        true_negatives  = true_negatives[indices]

        # compute false positives and true positives (Esto es lo mismo que sumar unos y ceros de cada una de los vectores pero se hace así para computar el AP)
        false_positives = np.cumsum(false_positives)
        true_positives  = np.cumsum(true_positives)
        true_negatives  = np.cumsum(true_negatives)

        # compute recall and precision (Y el F1)
        recall    = true_positives / num_annotations # Es lo mismo que dividir entre TP + FN porque la suma de ambas tiene que ser el número de anotaciones (se detecten o no)
        precision = true_positives / np.maximum(true_positives + false_positives, np.finfo(np.float64).eps)
        f1 = 2 * (precision * recall) / (precision + recall)

        # compute average precision
        average_precision  = compute_ap(recall, precision)
        average_precisions.append({'label': generator.labels[label], 'AP': average_precision, 'recall': recall[-1] if len(recall) else -1, 'precision': precision[-1] if len(precision) else -1, 'f1': f1[-1] if len(f1) else -1, 'support': num_annotations, 'TP':true_positives[-1], 'FP': false_positives[-1], 'TN': true_negatives[-1], 'FN': annotations_pending})

    return average_precisions

# Evaluación

## Carga de modelo y de datos de test

In [27]:
os.chdir('/home/ubuntu/tfm')
config_path = './utils/config.json'

with open(config_path) as config_buffer:    
    config = json.loads(config_buffer.read())

instances = pckl.load(open(config['model']['dataset_folder'], 'rb'))
labels = config['model']['labels']
labels = sorted(labels)

valid_generator = BatchGenerator(
    instances           = instances,
    anchors             = config['model']['anchors'],   
    labels              = sorted(config['model']['labels']),
)

infer_model = load_model(config['train']['model_folder'], config['train']['classes_path'], config['train']['anchors_path'])

/home/ubuntu/tfm/standalone/models/82f298920f8646ef89bd4313ef8202d9/artifacts/model/data/model.h5 model, anchors, and classes loaded in 18.03sec.


## Test

In [28]:
all_detections, all_annotations = detection(infer_model, valid_generator)

(416, 416, 3)
Found 2 boxes for img
Panel 0.96 (0, 474) (1080, 852)
Dedo 0.90 (472, 760) (649, 1103)
Time spent: 4.222sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.98 (211, 12) (1686, 661)
Dedo 0.81 (874, 492) (1027, 838)
Time spent: 0.476sec
(416, 416, 3)
Found 1 boxes for img
Dedo 0.96 (942, 534) (1067, 801)
Time spent: 0.419sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.98 (143, 98) (1709, 453)
Dedo 0.94 (850, 333) (1003, 629)
Time spent: 0.421sec
(416, 416, 3)
Found 1 boxes for img
Dedo 0.75 (719, 372) (1000, 663)
Time spent: 0.410sec
(416, 416, 3)
Found 3 boxes for img
Panel 0.62 (513, 971) (940, 1149)
Panel 0.99 (31, 553) (1027, 860)
Dedo 0.94 (408, 789) (623, 1225)
Time spent: 0.424sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.96 (0, 697) (1080, 1014)
Dedo 0.85 (396, 925) (578, 1174)
Time spent: 0.417sec
(416, 416, 3)
Found 1 boxes for img
Panel 0.55 (319, 337) (1004, 570)
Time spent: 0.404sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.75 (35, 317) (1799, 648)
Dedo 0.97 (9

Found 2 boxes for img
Panel 0.96 (0, 719) (1080, 1034)
Dedo 0.95 (461, 910) (590, 1225)
Time spent: 0.416sec
(416, 416, 3)
Found 3 boxes for img
Panel 0.50 (1073, 510) (1226, 695)
Panel 0.98 (341, 0) (1549, 406)
Dedo 0.90 (809, 250) (997, 502)
Time spent: 0.415sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.97 (314, 168) (1709, 506)
Dedo 0.93 (921, 459) (1085, 738)
Time spent: 0.412sec
(416, 416, 3)
Found 1 boxes for img
Dedo 0.90 (749, 437) (924, 730)
Time spent: 0.411sec
(416, 416, 3)
Found 1 boxes for img
Dedo 0.84 (873, 410) (1136, 721)
Time spent: 0.463sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.95 (0, 517) (1080, 856)
Dedo 0.97 (415, 758) (574, 1051)
Time spent: 0.552sec
(416, 416, 3)
Found 3 boxes for img
Panel 0.70 (1487, 913) (1661, 1026)
Panel 0.84 (854, 149) (1035, 465)
Dedo 0.95 (708, 452) (955, 888)
Time spent: 0.613sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.96 (465, 90) (1801, 455)
Dedo 0.93 (1124, 338) (1266, 604)
Time spent: 0.576sec
(416, 416, 3)
Found 2 boxes

Found 2 boxes for img
Panel 0.98 (464, 149) (1520, 400)
Dedo 0.85 (878, 314) (1080, 563)
Time spent: 0.409sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.91 (486, 196) (1405, 481)
Dedo 0.95 (939, 390) (1095, 671)
Time spent: 0.409sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.99 (112, 556) (1080, 895)
Dedo 0.97 (860, 778) (977, 1054)
Time spent: 0.415sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.90 (0, 0) (1920, 537)
Dedo 0.70 (921, 256) (1119, 708)
Time spent: 0.409sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.99 (0, 605) (1080, 974)
Dedo 0.97 (368, 828) (488, 1116)
Time spent: 0.414sec
(416, 416, 3)
Found 3 boxes for img
Panel 0.72 (189, 93) (426, 315)
Panel 0.96 (456, 124) (1762, 445)
Dedo 0.89 (942, 326) (1099, 653)
Time spent: 0.420sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.98 (7, 626) (1002, 985)
Dedo 0.96 (519, 869) (764, 1281)
Time spent: 0.424sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.90 (735, 0) (1734, 436)
Dedo 0.98 (1197, 252) (1347, 482)
Time spent: 0.406sec

Found 2 boxes for img
Panel 0.97 (329, 180) (1599, 426)
Dedo 0.98 (994, 315) (1117, 641)
Time spent: 0.419sec
(416, 416, 3)
Found 3 boxes for img
Panel 0.63 (1484, 908) (1664, 1027)
Panel 0.91 (859, 161) (1032, 447)
Dedo 0.97 (711, 428) (994, 880)
Time spent: 0.413sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.62 (20, 41) (161, 335)
Panel 0.79 (990, 247) (1721, 558)
Time spent: 0.421sec
(416, 416, 3)
Found 1 boxes for img
Panel 0.98 (0, 529) (1080, 896)
Time spent: 0.409sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.94 (0, 452) (921, 779)
Dedo 0.88 (438, 641) (621, 1162)
Time spent: 0.421sec
(416, 416, 3)
Found 1 boxes for img
Panel 0.99 (0, 688) (1080, 1040)
Time spent: 0.407sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.52 (1021, 266) (1267, 418)
Dedo 0.83 (574, 391) (727, 661)
Time spent: 0.408sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.91 (256, 163) (1711, 401)
Dedo 0.96 (1020, 292) (1178, 601)
Time spent: 0.417sec
(416, 416, 3)
Found 4 boxes for img
Panel 0.81 (11, 157) (32

Found 2 boxes for img
Panel 0.96 (666, 295) (1792, 615)
Dedo 0.95 (1129, 518) (1284, 803)
Time spent: 0.407sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.91 (0, 152) (1899, 702)
Dedo 0.98 (912, 482) (1040, 774)
Time spent: 0.417sec
(416, 416, 3)
Found 1 boxes for img
Panel 0.68 (362, 322) (491, 413)
Time spent: 0.405sec
(416, 416, 3)
Found 3 boxes for img
Panel 0.69 (0, 418) (486, 943)
Panel 0.96 (507, 251) (1430, 526)
Dedo 0.97 (936, 462) (1099, 706)
Time spent: 0.412sec
(416, 416, 3)
Found 1 boxes for img
Dedo 0.95 (707, 454) (965, 886)
Time spent: 0.406sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.98 (75, 243) (1282, 491)
Dedo 0.96 (737, 347) (869, 693)
Time spent: 0.424sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.80 (0, 192) (1920, 495)
Dedo 0.87 (1041, 366) (1159, 728)
Time spent: 0.407sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.79 (0, 303) (1920, 647)
Dedo 0.95 (1014, 536) (1154, 818)
Time spent: 0.406sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.96 (51, 244) (1323, 4

Found 1 boxes for img
Dedo 0.95 (734, 964) (839, 1272)
Time spent: 0.408sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.98 (0, 58) (1920, 749)
Dedo 0.97 (963, 486) (1132, 710)
Time spent: 0.419sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.96 (58, 249) (1301, 495)
Dedo 0.95 (713, 379) (825, 679)
Time spent: 0.413sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.97 (0, 691) (719, 905)
Dedo 0.96 (387, 880) (504, 1225)
Time spent: 0.424sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.99 (0, 733) (1057, 1081)
Dedo 0.97 (589, 947) (712, 1294)
Time spent: 0.425sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.98 (344, 129) (1647, 477)
Dedo 0.93 (1035, 428) (1197, 703)
Time spent: 0.418sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.97 (237, 297) (1909, 608)
Dedo 0.97 (975, 507) (1108, 809)
Time spent: 0.408sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.98 (581, 177) (1682, 523)
Dedo 0.93 (989, 392) (1182, 722)
Time spent: 0.408sec
(416, 416, 3)
Found 3 boxes for img
Panel 0.68 (287, 270) (924, 

Found 2 boxes for img
Panel 0.99 (294, 574) (1080, 859)
Dedo 0.95 (834, 724) (958, 1066)
Time spent: 0.437sec
(416, 416, 3)
Found 1 boxes for img
Panel 0.96 (202, 16) (1679, 645)
Time spent: 0.447sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.99 (596, 288) (1608, 558)
Dedo 0.88 (933, 511) (1115, 797)
Time spent: 0.406sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.99 (28, 583) (1051, 972)
Dedo 0.97 (505, 820) (630, 1123)
Time spent: 0.423sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.94 (223, 230) (1446, 490)
Dedo 0.98 (772, 383) (896, 659)
Time spent: 0.407sec
(416, 416, 3)
Found 3 boxes for img
Panel 0.71 (188, 337) (619, 550)
Panel 0.85 (620, 206) (1920, 479)
Dedo 0.97 (1274, 422) (1424, 733)
Time spent: 0.410sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.52 (985, 264) (1402, 422)
Dedo 0.98 (1016, 563) (1274, 1028)
Time spent: 0.409sec
(416, 416, 3)
Found 1 boxes for img
Dedo 0.89 (770, 334) (946, 715)
Time spent: 0.428sec
(416, 416, 3)
Found 3 boxes for img
Panel 0.55 (0, 323) (

In [29]:
average_precisions = evaluation(all_detections, all_annotations, valid_generator)

## Procesar salida

In [30]:
items = 0
precision = 0
for average_precision in average_precisions:    
    items += 1 
    precision += average_precision['AP']
display(pd.DataFrame(average_precisions))
print('mAP: {:.4f}'.format(precision / items))

,label,AP,recall,precision,support,TP,FP,FN
0,Dedo,0.784137,0.865169,0.839237,356.0,308.0,59.0,48.0
1,Panel,0.623460,0.681034,0.812757,580.0,395.0,91.0,185.0


mAP: 0.7038


## Experimental

In [31]:
average_precisions = evaluation_v2(all_detections, all_annotations, valid_generator)

In [32]:
items = 0
precision = 0
for average_precision in average_precisions:    
    items += 1 
    precision += average_precision['AP']
display(pd.DataFrame(average_precisions))
print('mAP: {:.4f}'.format(precision / items))

,label,AP,recall,precision,f1,support,TP,FP,TN,FN
0,Dedo,0.784137,0.865169,0.839237,0.852006,356.0,308.0,59.0,0.0,48.0
1,Panel,0.623460,0.681034,0.812757,0.741088,580.0,395.0,91.0,0.0,185.0


mAP: 0.7038


## Prueba completa

In [33]:
import mlflow
import os
import shutil
import boto3
from datetime import datetime


S3_CLIENT = boto3.resource('s3')

mlflow.set_tracking_uri(os.getenv('MLFLOW_TRACKING_URI'))
MLFLOW_CLIENT = mlflow.tracking.MlflowClient()

REGISTERED_MODELS = ["Hands"]
MODELS = {}


def downlod_model(bucket_name, remoteDirectory_name):

    bucket = S3_CLIENT.Bucket(bucket_name)

    for obj in bucket.objects.filter(Prefix=remoteDirectory_name):
        if not os.path.exists(os.path.dirname(obj.key)):
            os.makedirs(os.path.dirname(obj.key))
        bucket.download_file(obj.key, obj.key)


def update_models(version=-1, remove_old_versions=False):

    update = {}

    for model_name in REGISTERED_MODELS:
        model = None
        update[model_name] = 0
        for mv in MLFLOW_CLIENT.search_model_versions(f"name='{model_name}'"):
            mv_bckp = mv
            mv = dict(mv)
            if version == mv['version'] or (version == -1 and mv['current_stage'] == 'Production'):
                mv['last_updated_timestamp'] = str(datetime.fromtimestamp(int(mv['last_updated_timestamp'] / 1000)))
                bucket = mv['source'].split('//')[1].split('/')[0]
                folder = mv['source'].split('//')[1].split('/')[1]
                if os.path.exists(os.path.join('./models', folder)):
                    print("Load existing model...")
                    model = os.path.join(os.path.join('./models', folder), "artifacts/model/data/model.h5")
                else:
                    print("Downloading model...")
                    downlod_model(bucket, folder)
                    model = os.path.join(os.path.join('./models', folder), "artifacts/model/data/model.h5")
                    if remove_old_versions and os.path.exists('./models'):
                        shutil.rmtree('./models')
                    if not os.path.exists('./models'):
                        os.mkdir('./models')
                    shutil.move(os.path.join(os.getcwd(), folder), './models')
                    update[model_name] = 1
                print("Using model {name} v{version} ({current_stage}) updated at {last_updated_timestamp}".format(**mv))
                #response = {k: v for k, v in mv.items() if v}
                break
        if model:
            MODELS[model_name] = (model, mv_bckp)

    return update


def get_model(model_name):

    return MODELS.get(model_name, None)

In [ ]:
%%capture
os.chdir('/home/ubuntu/tfm/standalone')
config_path = '../utils/config.json'

with open(config_path) as config_buffer:    
    config = json.loads(config_buffer.read())

instances = pckl.load(open(config['model']['dataset_folder'], 'rb'))
labels = config['model']['labels']
labels = sorted(labels)

valid_generator = BatchGenerator(
    instances           = instances,
    anchors             = config['model']['anchors'],   
    labels              = sorted(config['model']['labels']),
)

versions = range(16,21)
for version in tqdm(versions):
    update_models(version)
    model_path, model_meta = get_model('Hands')
    infer_model = load_model(model_path, config['train']['classes_path'], config['train']['anchors_path'])
    all_detections, all_annotations = detection(infer_model, valid_generator)
    average_precisions = evaluation(all_detections, all_annotations, valid_generator)
    items = 0
    precision = 0
    for average_precision in average_precisions:    
        items += 1 
        precision += average_precision['AP']
    pckl.dump(((version,MLFLOW_CLIENT.get_run(model_meta.run_id)),(all_detections, all_annotations), (pd.DataFrame(average_precisions), 'mAP: {:.4f}'.format(precision / items))), open(f"{version}.pckl", 'wb'))

In [14]:
import os
os.chdir('/home/ubuntu/tfm/standalone')

/home/ubuntu/miniconda3/envs/tfm/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
import pickle as pckl
pckl.load(open('21_day.pckl', 'rb'))

/home/ubuntu/miniconda3/envs/tfm/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


((21,
  <Run: data=<RunData: metrics={'loss': 23.7419548034668,
   'lr': 1e-07,
   'stopped_epoch': 97.0,
   'val_loss': 23.5143699645996}, params={'augmentation': 'True',
   'baseline': 'None',
   'batch_size': '16',
   'early_stop_patience': '5',
   'epochs': '50',
   'initial_epoch': '0',
   'learning_rate': '0.0001',
   'min_delta': '0',
   'monitor': 'val_loss',
   'normalize': 'True',
   'opt_amsgrad': 'False',
   'opt_beta_1': '0.9',
   'opt_beta_2': '0.999',
   'opt_decay': '0.0',
   'opt_epsilon': '1e-07',
   'opt_learning_rate': '0.0001',
   'opt_name': 'Adam',
   'patience': '5',
   'reduce_lr_factor': '0.1',
   'reduce_lr_patience': '3',
   'restore_best_weights': 'False',
   'standarize': 'True',
   'steps_per_epoch': '28',
   'validation_split': '0.2',
   'validation_steps': '7'}, tags={'dataset': '/content/drive/My Drive/Colab '
              'Notebooks/TFM/TrainYourOwnYOLO/Data/Source_Images/Training_Images/vott-csv-export-augmented-all',
   'mlflow.log-model.history': 

In [72]:
rows = []
m = {'.pckl':'all', '_day.pckl': 'day', '_night.pckl': 'night'}
for s in ['.pckl', '_day.pckl', '_night.pckl']:
    for day in range(13, 22):
        data = pckl.load(open(str(day)+s, 'rb'))
        row ={
            'version': data[0][0],
            'mlflow': data[0][1],
            'result': data[2][0],
            'mAP': data[2][1].replace('mAP: ', '')
        }
        row['batch_size'] = row['mlflow'].data.params['batch_size']
        row['augmentation'] = row['mlflow'].data.params['augmentation']
        row['learning_date'] = float(row['mlflow'].data.params['learning_rate'])
        row['kind'] = m[s]
        del row['mlflow']
        rows.append(row)

In [105]:
final = []
for row in rows:
    first = f"{row['version']},{row['augmentation']},{row['batch_size']},{row['learning_date']},{row['kind']},{row['mAP']}"
    partial = []
    for _,r in row['result'].transpose().items():
        partial.append(','.join(list(map(lambda x:str(x), r))))
        #print(','.join(r[0]))
    second = ','.join(partial)
    final.append(','.join([first, second]))

In [106]:
for f in final:
    print(f)

13,False,4,0.0001,all,0.3812,Dedo,0.3829324831117263,0.43820224719101125,0.8082901554404145,356.0,156.0,37.0,200.0,Panel,0.3793993233311576,0.41551724137931034,0.8397212543554007,580.0,241.0,46.0,339.0
14,True,4,0.0001,all,0.7297,Dedo,0.8303311240100032,0.8932584269662921,0.895774647887324,356.0,318.0,37.0,38.0,Panel,0.6290790745162128,0.6741379310344827,0.8650442477876106,580.0,391.0,61.0,189.0
15,True,4,1e-06,all,0.0000,Dedo,0.0,-1,-1,356.0,-1,-1,356.0,Panel,0.0,-1,-1,580.0,-1,-1,580.0
16,True,4,1e-05,all,0.4879,Dedo,0.4488293183655009,0.5674157303370787,0.6917808219178082,356.0,202.0,90.0,154.0,Panel,0.5269231012964508,0.5775862068965517,0.7023060796645703,580.0,335.0,142.0,245.0
17,True,4,0.001,all,0.0135,Dedo,0.0,-1.0,-1.0,356.0,-1.0,-1.0,356.0,Panel,0.027079107505070994,0.027586206896551724,0.8888888888888888,580.0,16.0,2.0,564.0
18,True,8,0.0001,all,0.7246,Dedo,0.7902599471821179,0.851123595505618,0.8583569405099151,356.0,303.0,50.0,53.0,Panel,0.6589819074122285,0.68448275862068

In [101]:
s = []
for _,r in row['result'].transpose().items():
    s.append(','.join(list(map(lambda x:str(x), r))))
    #print(','.join(r[0]))
','.join(s)

'Dedo,0.8627183627965802,0.9127906976744186,0.9127906976744186,172.0,157.0,15.0,15.0,Panel,0.6597352011399578,0.6754966887417219,0.8947368421052632,302.0,204.0,24.0,98.0'

In [94]:
row['result']

,label,AP,recall,precision,support,TP,FP,FN
0,Dedo,0.862718,0.912791,0.912791,172.0,157.0,15.0,15.0
1,Panel,0.659735,0.675497,0.894737,302.0,204.0,24.0,98.0
